EvoGGen - Reproducing Behaviour via Evolutionary Grammar-Based Input Generation
===

**EvoGGen**, an acronym for Evolutionary Grammar-Based Input Generation, is an advanced extension of _EvoGFuzz_. It deploys evolutionary optimization strategies to systematically generate inputs that reproduce particular behaviours. The primary objective of EvoGGen is to learn a probabilistic grammar that encapsulates the characteristics of a user-specified behaviour. Consequently, it can generate new, unseen inputs that can trigger this defined behaviour, thereby expanding the boundaries of behaviour-oriented input generation.

Contrasting with EvoGFuzz: While the principal objective of EvoGFuzz is the detection of inputs inducing failures, EvoGGen aims for a different goal. It seeks to not only reproduce a predefined behaviour but also to generalize it, thereby broadening the range of inputs capable of triggering such behaviour.

In [1]:
import math

def calculator(inp: str) -> float:
    """
        A simple calculator function that can evaluate arithmetic expressions 
        and perform basic trigonometric functions and square root calculations.
    """
    return eval(
        str(inp), {"sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan}
    )

In [2]:
# Make sure you use the OracleResult from the evogfuzz library
from evogfuzz.oracle import OracleResult

def oracle(inp: str):
    """
    This function serves as an oracle or intermediary that catches and handles exceptions 
    generated by the 'calculator' function. The oracle function is used in the context of fuzz testing.
    It aims to determine whether an input triggers a bug in the 'calculator' function.

    Args:
        inp (str): The input string to be passed to the 'calculator' function.

    Returns:
        OracleResult: An enumerated type 'OracleResult' indicating the outcome of the function execution.
            - OracleResult.NO_BUG: Returned if the calculator function executes without any exception or only with CalculatorSyntaxError
            - OracleResult.BUG: Returned if the calculator function raises a ValueError exception, indicating a potential bug.
    """
    try:
        calculator(inp)
    except ValueError as e:
        return OracleResult.BUG
    
    return OracleResult.NO_BUG

In [3]:
initial_inputs = ['sqrt(-1)', 'cos(912)', 'tan(4)']

for inp in initial_inputs:
    print(inp.ljust(20), oracle(inp))

sqrt(-1)             BUG
cos(912)             NO_BUG
tan(4)               NO_BUG


In [4]:
from fuzzingbook.Grammars import Grammar, is_valid_grammar

CALCGRAMMAR: Grammar = {
    "<start>":
        ["<function>(<term>)"],

    "<function>":
        ["sqrt", "tan", "cos", "sin"],
    
    "<term>": ["-<value>", "<value>"], 
    
    "<value>":
        ["<integer>.<integer>",
         "<integer>"],

    "<integer>":
        ["<digit><integer>", "<digit>"],

    "<digit>":
        ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
}
    
assert is_valid_grammar(CALCGRAMMAR)

In [5]:
from evogfuzz.evogfuzz_class import EvoGGen

egg = EvoGGen(
    grammar=CALCGRAMMAR,
    inputs=['sqrt(-1.2)'],
    oracle=oracle,
    transform_grammar=True,
    iterations=20
)

In [6]:
grammar, _ = egg.optimize()

In [7]:
from fuzzingbook.ProbabilisticGrammarFuzzer import ProbabilisticGrammarFuzzer

prob_fuzzer = ProbabilisticGrammarFuzzer(grammar)

for _ in range(10):
    inp = prob_fuzzer.fuzz()
    print(inp.ljust(20), oracle(inp))

sqrt(-2)             BUG
sqrt(-864.1)         BUG
sqrt(-578771.1)      BUG
sqrt(-1.2)           BUG
sqrt(-882.8942)      BUG
sqrt(-2)             BUG
sqrt(-5582.487)      BUG
sqrt(-62)            BUG
sqrt(-876672.8443)   BUG
sqrt(-3662.2)        BUG
